## Naive Bayes Classification with Taitanic
bv https://blog.sicara.com/naive-bayes-classifier-sklearn-python-example-tips-42d100429e44


In [1]:
!dir/w

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 9684-467A

 C:\Users\user\Documents\kaggle-titanic-master 디렉터리

[.]                      [..]                     [.ipynb_checkpoints]
[images]                 [input]                  titanic-solution.ipynb
Untitled.ipynb           
               2개 파일             573,252 바이트
               5개 디렉터리  175,508,742,144 바이트 남음


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [2]:
# Importing dataset
data = pd.read_csv("input/train.csv")
data[:5]

FileNotFoundError: [Errno 2] File b'input/train.csv' does not exist: b'input/train.csv'

### 2. 전처리
#### 1) 범주형 데이터의 전처리

In [ ]:
data["Embarked"].value_counts(dropna=False)

In [57]:
s=pd.Series([2,3,4])

In [ ]:
s.value_counts

In [61]:
suv_rate = data["Survived"].value_counts(normalize=True, dropna=False)
suv_rate

0    0.593838
1    0.406162
Name: Survived, dtype: float64

In [6]:
# Convert categorical variable to numeric
data["Sex_cleaned"]=np.where(data["Sex"]=="male",0,1)
data["Embarked_cleaned"]=np.where(data["Embarked"]=="S",0,
                                  np.where(data["Embarked"]=="C",1,
                                           np.where(data["Embarked"]=="Q",2,3)
                                          )
                                 )
data["Embarked_cleaned"].value_counts()

0    644
1    168
2     77
3      2
Name: Embarked_cleaned, dtype: int64

#### 2) feature 추출 및 결측치 제거

In [9]:
# Cleaning dataset of NaN
data=data[[
    "Survived",
    "Pclass",
    "Sex_cleaned",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_cleaned"
]].dropna(axis=0, how='any')

In [14]:
data[:5]

,Survived,Pclass,Sex_cleaned,Age,SibSp,Parch,Fare,Embarked_cleaned
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


#### 3) 데이터 나누기

In [15]:
# Split dataset in training and test datasets
X_train, X_test = train_test_split(data, test_size=0.5, stratify=data["Survived"])

### 2. Base 모델 생성

In [16]:
# Instantiate the classifier
gnb = GaussianNB()
used_features =[
    "Pclass",
    "Sex_cleaned",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_cleaned"
]

# Train classifier
gnb.fit(
    X_train[used_features].values,
    X_train["Survived"]
)
y_pred = gnb.predict(X_test[used_features])

# Print results
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(
          X_test.shape[0],
          (X_test["Survived"] != y_pred).sum(),
          100*(1-(X_test["Survived"] != y_pred).sum()/X_test.shape[0])
))

Number of mislabeled points out of a total 357 points : 83, performance 76.75%


### 3. 성능 개선
Fare 정보만을 사용하여 분류를 제한하자.

In [17]:
mean_survival=np.mean(X_train["Survived"])
mean_not_survival=1-mean_survival
print("Survival prob = {:03.2f}%, Not survival prob = {:03.2f}%"
      .format(100*mean_survival,100*mean_not_survival))

Survival prob = 40.62%, Not survival prob = 59.38%


Compute probability distribution function :
- P(fare| Survival = 0)
- P(fare| Survival = 1)

In [18]:
mean_fare_survived = np.mean(X_train[X_train["Survived"]==1]["Fare"])
std_fare_survived = np.std(X_train[X_train["Survived"]==1]["Fare"])
mean_fare_not_survived = np.mean(X_train[X_train["Survived"]==0]["Fare"])
std_fare_not_survived = np.std(X_train[X_train["Survived"]==0]["Fare"])

print("mean_fare_survived = {:03.2f}".format(mean_fare_survived))
print("std_fare_survived = {:03.2f}".format(std_fare_survived))
print("mean_fare_not_survived = {:03.2f}".format(mean_fare_not_survived))
print("std_fare_not_survived = {:03.2f}".format(std_fare_not_survived))

mean_fare_survived = 48.61
std_fare_survived = 64.16
mean_fare_not_survived = 22.28
std_fare_not_survived = 28.45


In [27]:
data["Fare"].describe()

count    714.000000
mean      34.694514
std       52.918930
min        0.000000
25%        8.050000
50%       15.741700
75%       33.375000
max      512.329200
Name: Fare, dtype: float64

In [3]:
plt.rcParams['figure.figsize'] = (12.0, 8.0)

plt.hist(X_train[X_train["Survived"]==1]["Fare"],
         label="train survived Fare", bins=50, color='r')
from scipy.stats import norm
mu = mean_fare_survived; std = std_fare_survived
rv = norm(mu, std)
xs = np.linspace(0,520,1000)
plt.plot(xs, rv.pdf(xs)*5000*suv_rate[1], 'g')

plt.hist(X_train[X_train["Survived"]==0]["Fare"],
         label="Train not_survived Fare", bins=50, color='b')
mu = mean_fare_not_survived; std = std_fare_not_survived
rv = norm(mu, std)
plt.plot(xs, rv.pdf(xs)*5000*suv_rate[0], 'k')
plt.legend()

NameError: name 'plt' is not defined